In [5]:
import timm
import torch
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.feature_extraction import get_graph_node_names

model = timm.create_model(
    'efficientnet_b0',
    pretrained=True,
    num_classes=10
)

encoder = create_feature_extractor(model, {'conv_head': 'features'})
out = encoder(torch.rand(1, 3, 224, 224))
print([(k, v.shape) for k, v in out.items()])

[('features', torch.Size([1, 1280, 7, 7]))]


In [1]:
import torch
from functools import partial

import torchvision.transforms.v2 as v2
import pytorch_lightning as pl

from dl_toolbox import datamodules
from dl_toolbox import modules
from dl_toolbox import networks

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tf = v2.Compose(
    [
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

datamodule = datamodules.Resisc(
    data_path='/data',
    train_tf=tf,
    test_tf=tf,
    merge='all45',
    batch_size=16,
    num_workers=5,
    pin_memory=False
)

In [3]:
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchmetrics as M
import matplotlib.pyplot as plt
from dl_toolbox.utils import plot_confusion_matrix
from pytorch_lightning.utilities import rank_zero_info
import torch.nn as nn
import math
from dl_toolbox.transforms import Mixup
import timm


class Classifier(pl.LightningModule):
    def __init__(
        self,
        encoder,
        in_channels,
        class_list,
        optimizer,
        scheduler,
        metric_ignore_index,
        one_hot,
        tta=None,
        sliding=None,
        *args,
        **kwargs
    ):
        super().__init__()
        self.class_list = class_list
        self.num_classes = len(class_list)
        
        self.network = timm.create_model(
            encoder,
            pretrained=True,
            num_classes=self.num_classes
        )
        self.encoder = self.network.blocks
        self.loss = torch.nn.CrossEntropyLoss(
            ignore_index=-1,
            reduction='mean',
            weight=None,
            label_smoothing=0.
        )
        self.optimizer = optimizer
        self.scheduler = scheduler
        
        self.tta = tta
        self.sliding = sliding
        self.one_hot = one_hot
        
        metric_args = {
            'task': 'multiclass',
            'num_classes': self.num_classes,
            'ignore_index': metric_ignore_index
        }
        self.val_accuracy = M.Accuracy(**metric_args)
        self.test_accuracy = M.Accuracy(**metric_args)
        self.val_cm = M.ConfusionMatrix(**metric_args, normalize='true')
        self.test_cm = M.ConfusionMatrix(**metric_args, normalize='true')
    
    def configure_optimizers(self):
        parameters = list(self.parameters())
        trainable_parameters = list(filter(lambda p: p.requires_grad, parameters))
        rank_zero_info(
            f"The model will start training with only {sum([int(torch.numel(p)) for p in trainable_parameters])} "
            f"trainable parameters out of {sum([int(torch.numel(p)) for p in parameters])}."
        )
        optimizer = self.optimizer(params=trainable_parameters)
        scheduler = self.scheduler(optimizer)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch"
            },
        }

    def forward(self, x, sliding=None, tta=None):
        if sliding is not None:
            auxs = [self.forward(aux, tta=tta) for aux in sliding(x)]
            return sliding.merge(auxs)
        elif tta is not None:
            auxs = [self.forward(aux) for aux in tta(x)]
            logits = self.forward(x)
            return torch.stack([logits] + self.tta.revert(auxs)).sum(dim=0)
        else:
            return self.network.forward(x)
    
    def to_one_hot(self, y):
        return torch.movedim(F.one_hot(y, self.num_classes),-1,1).float()
    
    def training_step(self, batch, batch_idx):
        batch = batch["sup"]
        x = batch["image"]
        y = batch["target"]
        if self.one_hot: y = self.to_one_hot(y)
        logits_x = self.forward(x, sliding=None, tta=None)
        loss = self.loss(logits_x, y)
        self.log(f"ce/train", loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        x = batch["image"]
        y = batch["target"]
        if self.one_hot: y = self.to_one_hot(y)
        logits_x = self.forward(x, sliding=self.sliding)                    
        loss = self.loss(logits_x, y)
        self.log(f"ce/val", loss)
        probs = logits_x.softmax(dim=1)
        pred_probs, preds = torch.max(probs, dim=1)
        self.val_accuracy.update(preds, y)
        self.val_cm.update(preds, y)
        
    def on_validation_epoch_end(self):
        val_accuracy = self.val_accuracy.compute()
        print(val_accuracy)
        self.log("accuracy/val", val_accuracy)
        confmat = self.val_cm.compute().detach().cpu()
        self.val_accuracy.reset()
        self.val_cm.reset()
        class_names = [l.name for l in self.class_list]
        logger = self.trainer.logger
        fs = 12 - 2*(self.num_classes//10)
        fig = plot_confusion_matrix(confmat, class_names, norm=None, fontsize=fs)
        logger.experiment.add_figure("confmat/val", fig, global_step=self.trainer.global_step)

    def test_step(self, batch, batch_idx):
        x = batch["image"]
        y = batch["target"]
        if self.one_hot: y = self.to_one_hot(y)
        logits_x = self.forward(x, sliding=self.sliding, tta=self.tta)
        loss = self.loss(logits_x, y)
        self.log(f"ce/test", loss)
        probs = logits_x.softmax(dim=1)
        pred_probs, preds = torch.max(probs, dim=1)
        self.test_accuracy.update(preds, y)
        self.test_cm.update(preds, y)
        
    def on_test_epoch_end(self):
        self.log("accuracy/test", self.test_accuracy.compute())
        confmat = self.test_cm.compute().detach().cpu()
        self.test_accuracy.reset()
        self.test_cm.reset()
        class_names = [l.name for l in self.class_list]
        logger = self.trainer.logger
        fs = 12 - 2*(self.num_classes//10)
        fig = plot_confusion_matrix(confmat, class_names, norm=None, fontsize=fs)
        logger.experiment.add_figure("confmat/test", fig, global_step=self.trainer.global_step)

    def predict_step(self, batch, batch_idx):
        x = batch["image"]
        logits_x = self.forward(x, sliding=self.sliding, tta=self.tta)
        return logits_x.softmax(dim=1)
    
module = Classifier(
    encoder='efficientnet_b0',
    class_list=datamodule.class_list,
    optimizer=partial(torch.optim.SGD, lr=0.01, momentum=0.9, weight_decay=0.0001),
    scheduler=partial(torch.optim.lr_scheduler.ConstantLR, factor=1),
    in_channels=3,
    metric_ignore_index=None,
    one_hot=False,
    tta=None,
    sliding=None,
)

In [ ]:
from dl_toolbox.callbacks import ProgressBar, Finetuning, Lora, TiffPredsWriter, CalibrationLogger

lora = Lora('encoder', 4, True)

trainer = pl.Trainer(
    accelerator='gpu',
    devices=1,
    max_steps=100000,
    limit_train_batches=1.,
    limit_val_batches=1.,
    callbacks=[lora, ProgressBar()]
)

trainer.fit(
    module,
    datamodule=datamodule,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
Missing logger folder: /d/pfournie/dl_toolbox/dl_toolbox/modules/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
The model will start training with only 510125 trainable parameters out of 4065193.

  | Name          | Type                      | Params
------------------------------------------------------------
0 | network       | EfficientNet              | 4.1 M 
1 | encoder       | Sequential                | 3.6 M 
2 | loss          | CrossEntropyLoss          | 0     
3 | val_accuracy  | MulticlassAccuracy        | 0     
4 | test_accuracy | MulticlassAccuracy        | 0     
5 | val_cm        | MulticlassConfusionMatrix | 0     
6 | test_cm

Sanity Checking DataLoader 0:  50%|██████████████████████████████████████████████▌                                              | 1/2 [00:00<00:00,  2.52it/s]

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Sanity Checking DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.81it/s]tensor(0., device='cuda:0')


/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: 1035 NaN values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1575/1575 [01:39<00:00, 15.84it/s, v_num=0]
Validation: |                                                                                                                           | 0/? [00:00<?, ?it/s]

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 12. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


tensor(0.8890, device='cuda:0')
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1575/1575 [01:10<00:00, 22.43it/s, v_num=0]
Validation: |                                                                                                                           | 0/? [00:00<?, ?it/s]tensor(0.9230, device='cuda:0')
Epoch 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1575/1575 [01:10<00:00, 22.40it/s, v_num=0]
Validation: |                                                                                                                           | 0/? [00:00<?, ?it/s]tensor(0.9259, device='cuda:0')
Epoch 3: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1575/1575 [01:10<00:00, 22.38it/s, v_num=0]
Validation: |                                                                                                  